<h1> NLP Lab 4 </h1>

****
NAME: SHREE PRASAD M   
ROLL NUMBER: CH.EN.U4AIE22050
****

In [1]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import random
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re

2024-12-20 09:47:03.732093: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:03.735818: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:03.744979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734688023.759392   17280 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734688023.763727   17280 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:47:03.787599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
import nltk
nltk.download('reuters')
nltk.download('punkt_tab')

[nltk_data] Downloading package reuters to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def readData():
    data = ['This is a dog', 'This is a cat', 'I love my cat', 'This is my name']
    dat = []
    for i in range(len(data)):
        for word in data[i].split():
            dat.append(word)
    print(dat)
    return dat

def createBigram(data):
    listOfBigrams = []
    bigramCounts = {}
    unigramCounts = {}
    for i in range(len(data) - 1):
        if i < len(data) - 1 and data[i + 1].islower():
            listOfBigrams.append((data[i], data[i + 1]))
            if (data[i], data[i + 1]) in bigramCounts:
                bigramCounts[(data[i], data[i + 1])] += 1
            else:
                bigramCounts[(data[i], data[i + 1])] = 1
            if data[i] in unigramCounts:
                unigramCounts[data[i]] += 1
            else:
                unigramCounts[data[i]] = 1
    return listOfBigrams, unigramCounts, bigramCounts

def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):
    listOfProb = {}
    for bigram in listOfBigrams:
        word1 = bigram[0]
        word2 = bigram[1]
        listOfProb[bigram] = (bigramCounts.get(bigram)) / (unigramCounts.get(word1))
    return listOfProb

if __name__ == '__main__':
    data = readData()
    listOfBigrams, unigramCounts, bigramCounts = createBigram(data)

    print("\nAll the possible Bigrams are")
    print(listOfBigrams)

    print("\nBigrams along with their frequency")
    print(bigramCounts)

    print("\nUnigrams along with their frequency")
    print(unigramCounts)

    bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

    print("\nBigrams along with their probability")
    print(bigramProb)
    
    inputList = "This is my cat"
    splt = inputList.split()
    outputProb1 = 1
    bilist = []

    for i in range(len(splt) - 1):
        bilist.append((splt[i], splt[i + 1]))

    print("\nThe bigrams in the given sentence are")
    print(bilist)
    
    for i in range(len(bilist)):
        if bilist[i] in bigramProb:
            outputProb1 *= bigramProb[bilist[i]]
        else:
            outputProb1 *= 0

['This', 'is', 'a', 'dog', 'This', 'is', 'a', 'cat', 'I', 'love', 'my', 'cat', 'This', 'is', 'my', 'name']

All the possible Bigrams are
[('This', 'is'), ('is', 'a'), ('a', 'dog'), ('This', 'is'), ('is', 'a'), ('a', 'cat'), ('I', 'love'), ('love', 'my'), ('my', 'cat'), ('This', 'is'), ('is', 'my'), ('my', 'name')]

Bigrams along with their frequency
{('This', 'is'): 3, ('is', 'a'): 2, ('a', 'dog'): 1, ('a', 'cat'): 1, ('I', 'love'): 1, ('love', 'my'): 1, ('my', 'cat'): 1, ('is', 'my'): 1, ('my', 'name'): 1}

Unigrams along with their frequency
{'This': 3, 'is': 3, 'a': 2, 'I': 1, 'love': 1, 'my': 2}

Bigrams along with their probability
{('This', 'is'): 1.0, ('is', 'a'): 0.6666666666666666, ('a', 'dog'): 0.5, ('a', 'cat'): 0.5, ('I', 'love'): 1.0, ('love', 'my'): 1.0, ('my', 'cat'): 0.5, ('is', 'my'): 0.3333333333333333, ('my', 'name'): 0.5}

The bigrams in the given sentence are
[('This', 'is'), ('is', 'my'), ('my', 'cat')]


In [3]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for the model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurrence
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1

# Transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

dict(model["today", "the"])

{'public': 0.05555555555555555,
 'European': 0.05555555555555555,
 'Bank': 0.05555555555555555,
 'price': 0.1111111111111111,
 'emirate': 0.05555555555555555,
 'overseas': 0.05555555555555555,
 'newspaper': 0.05555555555555555,
 'company': 0.16666666666666666,
 'Turkish': 0.05555555555555555,
 'increase': 0.05555555555555555,
 'options': 0.05555555555555555,
 'Higher': 0.05555555555555555,
 'pound': 0.05555555555555555,
 'Italian': 0.05555555555555555,
 'time': 0.05555555555555555}

In [4]:
import random

# Starting words
text = ["today", "the"]
sentence_finished = False

while not sentence_finished:
    # Select a random probability threshold
    r = random.random()
    accumulator = 0.0

    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
        # Select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            break

    if text[-2:] == [None, None]:
        sentence_finished = True

# Join and print the generated text
print(' '.join([t for t in text if t]))

today the price of 12 , 000 , respectively , for example , on sale of 7 , 446 , 958 , 423 vs 2 , 723 Sales 10 . 9 IN 1985 / 86 .


In [5]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re

data_text = """The unanimous Declaration of the thirteen united States of America, When in
We hold these truths to be self-evident, that all men are created equal, that they are endou
He has refused his Assent to Laws, the most wholesome and necessary for the public good.
He has forbidden his Governors to pass Laws of immediate and pressing importance, unless su:
He has refused to pass other Laws for the accommodation of large districts of people, unles:
He has called together legislative bodies at places unusual, uncomfortable, and distant fror
He has dissolved Representative Houses repeatedly, for opposing with manly firmness his inv
He has refused for a long time, after such dissolutions, to cause others to be elected; whel
He has endeavoured to prevent the population of these States; for that purpose obstructing 1
He has obstructed the Administration of Justice, by refusing his Assent to Laws for establi:
He has made Judges dependent on his Will alone, for the tenure of their offices, and the amc
He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass oul
He has kept among us, in times of peace, Standing Armies without the Consent of our legislat
He has affected to render the Military independent of and superior to the Civil power.
He has combined with others to subject us to a jurisdiction foreign to our constitution, an
For Quartering large bodies of armed troops among us:
For protecting them, by a mock Trial, from punishment for any Murders which they should comr
For cutting off our Trade with all parts of the world:
For imposing Taxes on us without our Consent:
For depriving us in many cases, of the benefits of Trial by Jury:
For transporting us beyond Seas to be tried for pretended offences
For abolishing the free System of English Laws in a neighbouring Province, establishing thel
For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally
For suspending our own Legislatures, and declaring themselves invested with power to legisl
He has abdicated Government here, by declaring us out of his Protection and waging War agair
He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of (
He is at this time transporting large Armies of foreign Mercenaries to compleat the works o
He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against 1
He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhab:
In every stage of these Oppressions We have Petitioned for Redress in the most humble terms
Nor have We been wanting in attentions to our Brittish brethren. We have warned them from t:
We, therefore, the Representatives of the united States of America, in General Congress, As:"""

2024-12-31 13:06:54.145554: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 13:06:54.368948: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 13:06:54.561256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735650414.784253    3083 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735650414.925484    3083 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 13:06:55.554889: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [6]:
import re

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b", "", newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    long_words = []
    for i in newString.split():
        if len(i) >= 3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_new = text_cleaner(data_text)

In [7]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences
sequences = create_seq(data_new)
# Total Sequences: 7052

chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq, mapping):
    encoded_seqs = []
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        encoded_seqs.append(encoded_seq)
    return np.array(encoded_seqs)

# encode the sequences
sequences = encode_seq(sequences, mapping)

Total Sequences: 2366


In [8]:
sequences

array([[20,  8,  5, ...,  5,  0, 20],
       [ 8,  5,  0, ...,  0, 20,  8],
       [ 5,  0, 21, ..., 20,  8,  9],
       ...,
       [ 4,  0, 19, ...,  7, 18,  5],
       [ 0, 19, 20, ..., 18,  5, 19],
       [19, 20,  1, ...,  5, 19, 19]])

In [ ]:
# create X and y
X, y = sequences[:, :-1], sequences[:, -1]

# one hot encode y
y = to_categorical(y, num_classes=vocab)

In [11]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)

sequences = np.array(sequences)

# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (2129, 30) Val shape: (237, 30)


In [12]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

# Reshape sequences to make it 2-dimensional
sequences = sequences.reshape((len(sequences), sequences.shape[1]))

# create X and y
X, y = sequences[:, :-1], sequences[:, -1]

# one hot encode y
y = to_categorical(y, num_classes=vocab)

# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (2129, 30) Val shape: (237, 30)


In [13]:
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-12-31 13:08:20.133255: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
67/67 - 4s - 65ms/step - acc: 0.1348 - loss: 2.9806 - val_acc: 0.1772 - val_loss: 2.8968
Epoch 2/100
67/67 - 2s - 30ms/step - acc: 0.1982 - loss: 2.7097 - val_acc: 0.2700 - val_loss: 2.5975
Epoch 3/100
67/67 - 2s - 29ms/step - acc: 0.2654 - loss: 2.4433 - val_acc: 0.2658 - val_loss: 2.4505
Epoch 4/100
67/67 - 2s - 34ms/step - acc: 0.2950 - loss: 2.3215 - val_acc: 0.2785 - val_loss: 2.3988
Epoch 5/100
67/67 - 3s - 42ms/step - acc: 0.3288 - loss: 2.2533 - val_acc: 0.2911 - val_loss: 2.3669
Epoch 6/100
67/67 - 2s - 33ms/step - acc: 0.3410 - loss: 2.1939 - val_acc: 0.3038 - val_loss: 2.3138
Epoch 7/100
67/67 - 2s - 24ms/step - acc: 0.3481 - loss: 2.1387 - val_acc: 0.3418 - val_loss: 2.2771
Epoch 8/100
67/67 - 3s - 39ms/step - acc: 0.3645 - loss: 2.0856 - val_acc: 0.3291 - val_loss: 2.2599
Epoch 9/100
67/67 - 3s - 39ms/step - acc: 0.3805 - loss: 2.0299 - val_acc: 0.3249 - val_loss: 2.2328
Epoch 10/100
67/67 - 2s - 37ms/step - acc: 0.3894 - loss: 1.9849 - val_acc: 0.3207 - v

In [14]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict(encoded, verbose=0)
        # get the index of the character with the highest probability
        yhat = np.argmax(yhat)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += out_char
    return in_text

inp = 'Large armies of'
print(len(inp))
print(generate_seq(model, mapping, 50, inp.lower(), 15))

15
large armies offices and sent 
